In [4]:
from PIL import ImageOps, Image


In [5]:
img = Image.open("blandwh.png")
img.show()

In [102]:
num = 1
map = {}
indexes = {}
for i in range(1,img.height,100):
    for j in range(1, img.width,100):
        border = (1+i, 1+j, 99+i, 99+j)
        ims = img.crop(border)
        ims.save("CroppedImages/"+str(num)+".png")
        text = pytesseract.image_to_string(Image.open('CroppedImages/'+str(num)+'.png'))
        if (map.get(text) == None):
            map[text] = 1
            indexes[text] =[num]
        else:
            map[text] +=1
            indexes[text].append(num)
        num +=1 

In [152]:
fixedMap = {}
fixedIndexes = {}
fixedMap["Black_"] = 0
fixedMap["White_"] = 0
fixedIndexes["Black_"] = []
fixedIndexes["White_"] = []
for i in map.keys():
    newkey = (re.sub("[\n\(\)\,]",' ', i))
    if(re.search("[0-9]+",newkey)):
        sum = int(reduce(lambda x, y: int(x)+int(y), re.findall("[0-9]+",newkey)))
        if(sum>150):
            fixedMap["White_"]+=map[i]
            fixedIndexes["White_"]+=(indexes[i])
        else:
            fixedMap["Black_"]+=map[i]
            fixedIndexes["Black_"]+=(indexes[i])
    else:
        if (fixedMap.get(newkey) == None):
            fixedMap[newkey] = map[i]
            fixedIndexes[newkey] = (indexes[i])

        else:
            fixedMap[newkey] +=map[i]
            fixedIndexes[newkey]+=(indexes[i])


In [134]:
Black = ['Black_',  'Black', 'Blackest cell ever', 'Very black', 'Bis for black', 'Darkness', 'Not white', 'Blackest cellever',  '‘Blackest cell ever', 'very black', '']

In [166]:
for i in fixedMap.keys():
    to = ""
    if(i in Black):
        to = "Black/"
    else:
        to = "White/"
    for file in fixedIndexes[i]:
        try:
            os.rename("CroppedImages/"+str(file)+".png",to+str(file)+".png") 
        except WindowsError:
            o =2
        

In [8]:
fixedIndexes

NameError: name 'fixedIndexes' is not defined

In [130]:
fixedMap.keys()

dict_keys(['Black_', 'White_', 'Black', 'Blackest cell ever', 'Very black', 'Bis for black', 'Darkness', 'White', 'RtGt+B', 'R+G+B', 'Not white', 'This one is also white', 'Blackest cellever', 'white', 'Wis for white', 'Simply white', '‘Blackest cell ever', 'very black', 'Rtagte', 'R+etB', 'Wig for white', 'w is for white', ''])

In [117]:
indexes

{'Black': [1,
  16,
  34,
  61,
  68,
  79,
  94,
  111,
  123,
  137,
  139,
  142,
  144,
  151,
  159,
  172,
  177,
  235,
  254,
  259,
  269,
  272,
  275,
  313,
  333,
  334,
  386,
  410,
  412,
  464,
  465,
  470,
  487,
  490,
  493,
  522,
  524,
  535,
  536,
  542,
  556,
  564,
  576,
  597,
  613,
  621,
  632,
  638,
  656,
  674,
  690,
  692,
  704,
  711,
  714,
  723,
  753,
  761,
  770,
  781,
  804,
  818,
  859,
  860,
  876,
  879,
  882,
  894,
  900,
  908,
  909,
  923,
  943,
  967,
  973,
  977,
  1002,
  1022,
  1027,
  1031,
  1046,
  1047,
  1050,
  1053,
  1056,
  1059,
  1063,
  1067,
  1102,
  1106,
  1112,
  1116,
  1122,
  1123,
  1156,
  1180,
  1185,
  1189,
  1195,
  1196,
  1217,
  1222,
  1259,
  1263,
  1267,
  1284,
  1312,
  1335,
  1338,
  1342,
  1352,
  1356,
  1359],
 'Blackest\ncell ever': [2,
  9,
  19,
  36,
  44,
  51,
  85,
  95,
  145,
  152,
  158,
  268,
  287,
  370,
  375,
  398,
  422,
  434,
  436,
  451,
  468,
  494,
  5

In [113]:
105+113+50+75+112+69+78+19+30+19+4

674

In [115]:
reduce(lambda x,y:x+y,fixedMap.values())

1369

In [2]:
Image

NameError: name 'Image' is not defined

In [9]:
a = [0]*1369

In [29]:
for i in range(1369):
    try:
        Image.open("Black/"+str(i+1)+".png")
        a[i] = 1
    except FileNotFoundError:
        a[i] = 0

In [36]:
f = open("demofile2.txt", "w")
for i in range(37):
    for j in range(37):
        f.write(str(a[j*37+i])+",")
    f.write("\n")
f.close()

In [39]:
from functools import reduce
import re

In [42]:
int(reduce(lambda x, y: int(x)+int(y), re.findall("[0-9]+","A32B112D13G18I34M80O31R37T85U229V61W17X94Y47")))

890

In [ ]:
A28B111D13G18I33M76O31R37T87U229V58W16X90Y47